In [ ]:
!pip install langchain langchain-ollama ollama

In [ ]:
from langchain_ollama import OllamaLLM

llamamodel = OllamaLLM(model="llama3.3:latest")

result = llamamodel.invoke(input="hello")

print(result)

In [ ]:
!pip install transformers accelerate datasets sentence-transformers
!pip install pinecone-client sentence-transformers pandas

In [6]:
import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import pandas as pd


In [7]:
# Initialize Pinecone
pc = Pinecone(api_key="pcsk_6E9B6o_DHFJaybC7zzr4QT9i1tZo1vExTxji5j1syULud17p1HXAzrZPN7Zv4fs9H83L98")  # Replace with your Pinecone API key

index_name = "nyd"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Adjust based on your embedding dimension
        metric='cosine',  # Use cosine similarity
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")

Connected to index: nyd


In [8]:
# Load Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Load and preprocess the dataset
df = pd.read_csv("combined_file.csv")
df['Combined_Questions'] = df['question'].apply(lambda x: " ".join(x.split("?")).strip())
embeddings = model.encode(df['Combined_Questions'].tolist(), convert_to_numpy=True)

# Upload embeddings and metadata to Pinecone
for i, row in df.iterrows():
    metadata = {
        "answer": row['answer'],
        "chapter": row['chapter'],
        "verse": row['verse'],
        "sanskrit": row['sanskrit']
    }
    index.upsert([(str(i), embeddings[i].tolist(), metadata)])
print("Embeddings with metadata uploaded to Pinecone!")

In [9]:
# Function to retrieve answers and metadata from Pinecone
def retrieve_answer(input_question, top_k=1):
    query_embedding = model.encode(input_question, convert_to_numpy=True)
    result = index.query(
        vector=query_embedding.tolist(),
        top_k=top_k,
        include_metadata=True
    )
    answers = []
    for match in result['matches']:
        metadata = match['metadata']
        answers.append({
            "score": match['score'],
            "answer": metadata['answer'],
            "chapter": metadata.get('chapter', 'N/A'),
            "verse": metadata.get('verse', 'N/A'),
            "sanskrit": metadata.get('sanskrit', 'N/A')
        })
    return answers

In [10]:
def answer_query_from_llama(query):
    llama1result =llamamodel.invoke(input = query)
    return llama1result

In [12]:
# Llama Query Refinement
def refine_query_with_llama(query, retrieved_info):
    prompt_Template_refine=f""""You are an assistant specializing in refining queries for better retrieval.\n Original query: '{query}'\n Retrieved information:\n{retrieved_info}\n Refine the query to include specific details for improved results.\n If the query is already precise, return it unchanged. Refined query:"""
    Refine_result= llamamodel.invoke(input = prompt_Template_refine)
    return Refine_result

In [13]:
# Llama Final Response Generation
def generate_final_response_with_llama(query, retrieved_info, llm_retrieved):
    prompt_Template_Generate = f"""You are an expert at combining information to generate detailed answers.\n Original query: '{query}'\n Retrieved information from semantic search:\n{retrieved_info} \n Retrieved information from Llama:\n{llm_retrieved} \n Provide chapter, verse, sanskrit, traslation if the query is directly belongs to sanskrit. \n "If the query is not directly related to Bhagavad Gita and Pantanjali yoga sutra return 'This Question not directly related to Bhagavad Gita or Pantanjali yoga sutra' with no extra words"  \n Don't say how you process this context in the answer. \n Using all the provided context, generate a complete, accurate, and concise answer. \n  "Provide the output in the following dictionary format within double quotes for keys and values: \n\n{{\n  Book_name' : <Book_name>,\n  'chapter': <chapter_number>,\n  'verse': <verse_number>,\n  'sanskrit': <sanskrit_text>,\n  'Translation' : <Translation>,\n  'answer': <enlarge_answer>\n}}\n\n"""
    Generate_result = llamamodel.invoke(input =  prompt_Template_Generate)
    return Generate_result

In [ ]:
def final_resp(query_test):
  user_query =query_test


  # Step 1: Retrieve answer from Pinecone
  semantic_results = retrieve_answer(user_query, top_k=3)

  retrieved_info = "\n".join([
      f"Score: {item['score']}, Answer: {item['answer']}, Chapter: {item['chapter']}, Verse: {item['verse']}, Sanskrit: {item['sanskrit']}"
      for item in semantic_results
  ])

  # Step 2: Retrieve answer from Llama
  llm_result = answer_query_from_llama(user_query)

  # Step 3: Refine the query with Llama
  refined_query = refine_query_with_llama(user_query, retrieved_info)

  # Step 4: Generate final response using Llama
  final_response = generate_final_response_with_llama(refined_query, retrieved_info, llm_result)

  # Display the results
  print("=====================================================")
  print("Semantic Search Results:")
  print(retrieved_info)
  print("-----------------------------------------------------")
  print(llm_result)
  print("=====================================================")
  print(f"Refined Query: {refined_query}")
  print("-----------------------------------------------------")
  print(f"Final Response:\n{final_response}")
  print("=====================================================")
  return final_response


In [ ]:
import csv
import json
import os

query_test = input("Enter the Question: ")
# Get the response from the final_resp function 
final_dic = final_resp(query_test)
output_folder = "output_folder"

try:
  # Convert the JSON string to a Python dictionary
  parsed_dict = json.loads(final_dic)


# Save the parsed dictionary as a JSON file
  json_filename = f"{output_folder}/output_.json" 
  with open(json_filename, "w", encoding="utf-8") as json_file:
      json.dump(parsed_dict, json_file, ensure_ascii=False, indent=4)

  print(f"Saved JSON file for this Question : {json_filename}")
except json.JSONDecodeError as e:
  print(f"Error decoding JSON for this Question : {e}")
